In [5]:
import pandas as pd 
import numpy as np
import collections
pd.set_option('display.max_columns', None)

In [4]:
# first make bike accident dataset

fileName = "data/source data/accidents.csv"
usecolsall = ["X","Y","OBJECTID","CRIMEID","CCN","REPORTDATE","ROUTEID","MEASURE","OFFSET","STREETSEGID","ROADWAYSEGID","FROMDATE","TODATE","MARID","ADDRESS","LATITUDE","LONGITUDE","XCOORD","YCOORD","WARD","EVENTID","MAR_ADDRESS","MAR_SCORE","MAJORINJURIES_BICYCLIST","MINORINJURIES_BICYCLIST","UNKNOWNINJURIES_BICYCLIST","FATAL_BICYCLIST","MAJORINJURIES_DRIVER","MINORINJURIES_DRIVER","UNKNOWNINJURIES_DRIVER","FATAL_DRIVER","MAJORINJURIES_PEDESTRIAN","MINORINJURIES_PEDESTRIAN","UNKNOWNINJURIES_PEDESTRIAN","FATAL_PEDESTRIAN","TOTAL_VEHICLES","TOTAL_BICYCLES","TOTAL_PEDESTRIANS","PEDESTRIANSIMPAIRED","BICYCLISTSIMPAIRED","DRIVERSIMPAIRED","TOTAL_TAXIS","TOTAL_GOVERNMENT","SPEEDING_INVOLVED","NEARESTINTROUTEID","NEARESTINTSTREETNAME","OFFINTERSECTION","INTAPPROACHDIRECTION","LOCATIONERROR","LASTUPDATEDATE","MPDLATITUDE","MPDLONGITUDE","MPDGEOX","MPDGEOY","BLOCKKEY"]
accidentData = pd.read_csv(fileName, usecols=usecolsall)
accidentData['FROMDATE'] = pd.to_datetime(accidentData['FROMDATE']) 
accidentData = accidentData[accidentData['FROMDATE'] > "2016-12-31"]
accidentData['FROMDATE'] = pd.to_datetime(accidentData['FROMDATE']) 
accidentData = accidentData[accidentData['FROMDATE'] > "2016-12-31"]

smallData = accidentData[['TOTAL_BICYCLES','TOTAL_VEHICLES','TOTAL_PEDESTRIANS','LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'FATAL_BICYCLIST', 'MPDLATITUDE', 'MPDLONGITUDE', 'FROMDATE','STREETSEGID','ROUTEID','NEARESTINTSTREETNAME','NEARESTINTROUTEID','OFFINTERSECTION']]

# get only accidents that include a bike
bikeData = smallData[smallData['TOTAL_BICYCLES'] > 0]

# save to csv
bikeData.to_csv("data/modified data/bikes.csv", index=False)

ValueError: Usecols do not match columns, columns expected but not found: ['MARID', 'BLOCKKEY']

In [7]:
# now make bike share movement data
bikeShareTrips = pd.read_csv("data/source data/bikeShareTrips.csv")
bikeShareLocations = pd.read_csv("data/source data/bikeShareLocations.csv")
startLocations = bikeShareTrips['Start station'].values
endLocations = bikeShareTrips['End station'].values
allLocations = [i.lower().strip() for i in list(np.unique(np.concatenate((startLocations, endLocations), axis=0)))]
locationList = bikeShareLocations['ADDRESS'].apply(lambda x:x.lower().strip())

print(len(allLocations))
print(len(locationList))

b = 0
for i in locationList:
    if (i in allLocations):
        b = b + 1
print(b)
    
fr = []
for ss, es in zip(bikeShareTrips['Start station number'], bikeShareTrips['End station number']):
    fr.append(str(ss) + ' - ' + str(es))
bikeShareTrips['fullRoutes'] = fr

start_station_ids = []
end_station_ids = []
total_rides = []
start_station_lats = []
start_station_longs = []
end_station_lats = []
end_station_longs = []

for i,j in zip(bikeShareTrips['fullRoutes'].value_counts().index,bikeShareTrips['fullRoutes'].value_counts()):
    
    start_station_id = int((i.split("-")[0]).strip())
    start_station_ids.append(start_station_id)
    end_station_id = int((i.split("-")[1]).strip())
    end_station_ids.append(end_station_id)
    
    start_station_row = bikeShareLocations[bikeShareLocations['TERMINAL_NUMBER'] == start_station_id]
    end_station_row = bikeShareLocations[bikeShareLocations['TERMINAL_NUMBER'] == end_station_id]

    if (len(start_station_row['LATITUDE'].values) > 0):
        start_station_lats.append(start_station_row['LATITUDE'].values[0])
    else:
        start_station_lats.append(None)
        
    if (len(start_station_row['LONGITUDE'].values) > 0):
        start_station_longs.append(start_station_row['LONGITUDE'].values[0])
    else:
        start_station_longs.append(None)
        
    if (len(end_station_row['LATITUDE'].values) > 0):
        end_station_lats.append(end_station_row['LATITUDE'].values[0])
    else:
        end_station_lats.append(None)
        
    if (len(end_station_row['LONGITUDE'].values) > 0):
        end_station_longs.append(end_station_row['LONGITUDE'].values[0])
    else:
        end_station_longs.append(None)
    
    total_rides.append(j)
    
bikeShareMovement = pd.DataFrame(
            {
                "start_station_id" : start_station_ids,
                "end_station_id" : end_station_ids,
                "total_ride" : total_rides,
                "start_station_lat" : start_station_lats,
                "start_station_long" : start_station_longs,
                "end_station_lat" : end_station_lats,
                "end_station_long" : end_station_longs
            })

bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'].notna()]

bikeShareMovement.head()
bikeShareMovement.to_csv("data/modified data/bikeShareMovement.csv")

650
579
577


In [8]:
# now make bike total traffic data
bikeShareTrips = pd.read_csv("data/source data/bikeShareTrips.csv")
bikeShareLocations = pd.read_csv("data/source data/bikeShareLocations.csv")

locationInRides = []
locationOutRides = []
locationNames = []
locationIds = []
locationLats = []
locationLongs = []
locationBikes = []

startCounter = collections.Counter(bikeShareTrips['Start station number'])
endCounter = collections.Counter(bikeShareTrips['End station number'])

for location_id, location_name, location_lat, location_long, location_bikes in zip(bikeShareLocations["TERMINAL_NUMBER"],bikeShareLocations["ADDRESS"],bikeShareLocations["LATITUDE"],bikeShareLocations["LONGITUDE"],bikeShareLocations["NUMBER_OF_BIKES"]):
    startCount = 0
    endCount = 0
    startCount = startCounter.get(location_id)
    endCount = endCounter.get(location_id)
            
    locationInRides.append(startCount)
    locationOutRides.append(endCount)
    locationNames.append(location_name)
    locationIds.append(location_id)
    locationLats.append(location_lat)
    locationLongs.append(location_long)
    locationBikes.append(location_bikes)

bikeShareStationTotals = pd.DataFrame(
            {
                "station" : locationNames,
                "station_id" : locationIds,
                "total_out" : locationOutRides,
                "total_in" : locationInRides,
                "station_lat" : locationLats,
                "station_long" : locationLongs,
                "station_bikes" : locationBikes
            })

bikeShareStationTotals.to_csv("data/modified data/bikeShareStationTotals.csv")